In [2]:
import pandas as pd

In [9]:
data = pd.read_csv('../train_sentences.tsv', sep='\t', header=None, names=['e1_id', 'e2_id', 'rel_id', 'e1_label', 'e2_label', 'rel_label', 'sent'])

In [10]:
data

,e1_id,e2_id,rel_id,e1_label,e2_label,rel_label,sent
0,Q15711517,Q3133,P462,Granny Smith,green,color,Granny Smith apples are light green in colour.
1,Q5177331,Q838368,P462,Country Hotel,black-and-white,color,"For some reason, both he and Riam are rather g..."
2,Q1494793,Q22006653,P462,Garuda,color,color,"According to the text Silparatna , states Rao,..."
3,Q12105566,Q838368,P462,The Card,black-and-white,color,The black-and-white film The Card (retitled Th...
4,Q2056924,Q22006653,P462,A Dandy in Aspic,color,color,A Dandy in Aspic A Dandy in Aspic is a 1968 Te...
5,Q1035443,Q22006653,P462,Caravaggio,colour,color,Susinno's early-18th-century Le vite de' pitto...
6,Q830191,Q838368,P462,Gods of the Plague,black-and-white,color,Gods of the Plague Gods of the Plague () is a ...
7,Q1557904,Q22006653,P462,The Tingler,color,color,Although The Tingler was filmed in black-and-w...
8,Q1752154,Q1088,P462,Blastoise,blue,color,"In a poll conducted by IGN, it was voted as th..."
9,Q5463392,Q838368,P462,Flying Disc Man from Mars,black-and-white,color,Flying Disc Man from Mars Flying Disc Man from...


### Run Stanford CoreNLP

In [5]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [ ]:
openie_outputs = []
len_data = len(data['sent'])
for i in range(len_data):
    print(i, 'of', len_data - 1)
    corenlpres = nlp.annotate(data['sent'][i], properties={
            'ssplit.eolonly': True,
            'openie.triple.all_nominals': False,
            'openie.triple.strict': True,
            'openie.max_entailments_per_clause': 500,
            'annotators': 'openie',
            'outputFormat': 'json',
        })
    for o in corenlpres['sentences'][0]['openie']:
        if ((data['e1_label'][i] in o['subject'] or o['subject'] in data['e1_label'][i])
            and (data['e2_label'][i] in o['object'] or o['object'] in data['e2_label'][i])):
            # subject and object extracted by open IE contains the focus entity
            openie_outputs.append((1, i, o['subject'], o['relation'], o['object']))
        elif ((data['e1_label'][i] not in o['subject'] and o['subject'] not in data['e1_label'][i])
            or (data['e2_label'][i] not in o['object'] and o['object'] not in data['e2_label'][i])):
            openie_outputs.append((0, i, o['subject'], o['relation'], o['object']))

In [12]:
len(openie_outputs)

1946893

In [13]:
with open('../train_openie.tsv', 'w') as f:
    for item in openie_outputs:
        f.write(str(item[0]) + '\t' + str(item[1]) + '\t' + item[2] + '\t' + item[3] + '\t' + item[4] + '\n')
f.closed

True